In [36]:
import pandas
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.optimizers import Ftrl
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [132]:
# Вариант 3
# Цель 20/7 + 1 = 7

# X N(0,10)
# e N(0,0.3)
# Цель -X + e
x = [0, 10]
e = [0, 0.3]
dx = x[1]-x[0]

In [133]:
def function(xarr):
    return [-x + np.random.rand(1) * e[1] for x in xarr]

def encode(x):
    return x + 10
#     return x * 2 + 1

In [134]:
def genarate_data(count):
    xarr = [(np.random.rand(1) * (x[1]-x[0]))+x[0] for idx in range(count)]
    localVal = function(xarr)
    return np.hstack((np.asarray(xarr), np.asarray(localVal)))

In [135]:
np.savetxt("train.csv", genarate_data(800), delimiter=",")
np.savetxt("valid.csv", genarate_data(200), delimiter=",")

In [136]:
trainData = np.genfromtxt('train.csv', delimiter=',')
validData = np.genfromtxt('valid.csv', delimiter=',')

trainX = trainData[:, 1:]
trainVal = trainData[:, :1]
validX = validData[:, 1:]
validVal = validData[:, :1]
encodedTrainX = encode(trainX)
encodedValidX = encode(validX)

In [ ]:
input_layer = Input(shape=(1,), name='firstInput')
# encoding_layer = Dense(2)(input_layer)
encoding_output = Dense(1, name="encoding")(input_layer)
# decoding_layer = Dense(2)(encoding_output)
decoding_output = Dense(1, name="decoding")(encoding_output)
regression_layer = Dense(20, activation='relu')(encoding_output)
regression_layer = Dense(20, activation='relu')(regression_layer)
regression_output = Dense(1, name='regression')(regression_layer)

model = Model(inputs=[input_layer], outputs=[regression_output, encoding_output, decoding_output])
model.compile(optimizer='rmsprop', loss='mse', metrics='mae')
model.fit([trainX], [trainVal, encodedTrainX, trainX], epochs=100, batch_size=10, validation_split=0, verbose=True)

Epoch 1/100
80/80 [==============================] - 2s 7ms/step - loss: 47.2925 - regression_loss: 6.4440 - encoding_loss: 37.3614 - decoding_loss: 3.4870 - regression_mae: 1.9130 - encoding_mae: 5.2703 - decoding_mae: 1.5895
Epoch 2/100
80/80 [==============================] - 1s 7ms/step - loss: 38.2446 - regression_loss: 0.0135 - encoding_loss: 35.9261 - decoding_loss: 2.3051 - regression_mae: 0.0938 - encoding_mae: 5.1519 - decoding_mae: 1.2651
Epoch 3/100
80/80 [==============================] - 1s 7ms/step - loss: 35.6965 - regression_loss: 0.0112 - encoding_loss: 34.0779 - decoding_loss: 1.6074 - regression_mae: 0.0877 - encoding_mae: 5.0452 - decoding_mae: 1.0472
Epoch 4/100
80/80 [==============================] - 1s 7ms/step - loss: 32.3628 - regression_loss: 0.0113 - encoding_loss: 31.3720 - decoding_loss: 0.9795 - regression_mae: 0.0862 - encoding_mae: 4.8433 - decoding_mae: 0.7984
Epoch 5/100
80/80 [==============================] - 1s 7ms/step - loss: 30.7452 - regressio

In [ ]:
# encoding
encoding_model = Model(inputs=[input_layer], outputs=[encoding_output])
encoding_prediction = encoding_model.predict(validX)
encoding_model.save('encoding.h5')
np.savetxt('encoding.csv', np.hstack((encodedValidX, encoding_prediction)), delimiter=',')
# decoding
decoding_model = Model(inputs=[input_layer], outputs=[encoding_output])
decoding_prediction = decoding_model.predict(validX)
decoding_model.save('decoding.h5')
np.savetxt('decoding.csv', np.hstack((validX, decoding_prediction)), delimiter=',')
# regression
regression_model = Model(inputs=[input_layer], outputs=[encoding_output])
regression_prediction = regression_model.predict(validX)
regression_model.save('regression.h5')
np.savetxt('regression.csv', np.hstack((validVal, regression_prediction)), delimiter=',')